In [ ]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

In [2]:
proj_dir = Path("../../../")

In [3]:
bon_df = pd.read_csv(proj_dir / "Data/insitu/fish/processed/DART_BON.csv")
tda_df = pd.read_csv(proj_dir / "Data/insitu/fish/processed/DART_TDA.csv")
lyl_df = pd.read_csv(proj_dir / "Data/insitu/fish/processed/DART_LYL.csv")

In [4]:
bon_df['date'] = pd.to_datetime(bon_df['date'])
tda_df['date'] = pd.to_datetime(tda_df['date'])
lyl_df['date'] = pd.to_datetime(lyl_df['date'])

In [5]:
# filter out 2020 to 2023
bon_df_ = bon_df[(bon_df['date'].dt.year >= 2020) & (bon_df['date'].dt.year <= 2022)]
tda_df_ = tda_df[(tda_df['date'].dt.year >= 2020) & (tda_df['date'].dt.year <= 2022)]
lyl_df_ = lyl_df[(lyl_df['date'].dt.year >= 2020) & (lyl_df['date'].dt.year <= 2022)]
# bon_df_

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(15, 5), sharex=True)
bon_df_.plot(x='date', y=['chinook'], style='-', ax=ax1)
tda_df_.plot(x='date', y=['chinook'], style='-', ax=ax2)
lyl_df_.plot(x='date', y=['chinook'], style='-', ax=ax3)

In [8]:
fall_start_month = 8
fall_start_day = 1
fall_end_month = 11
fall_end_day = 15

In [9]:
bon_df['year'] = bon_df['date'].dt.year
bon_df['month'] = bon_df['date'].dt.month
bon_df['day'] = bon_df['date'].dt.day

tda_df['year'] = tda_df['date'].dt.year
tda_df['month'] = tda_df['date'].dt.month
tda_df['day'] = tda_df['date'].dt.day

lyl_df['year'] = lyl_df['date'].dt.year
lyl_df['month'] = lyl_df['date'].dt.month
lyl_df['day'] = lyl_df['date'].dt.day

In [ ]:
# group bon_df by year and plot
grouped = bon_df.groupby('year')

fig, axs = plt.subplots(len(grouped['year'].unique()), 1, figsize=(10, 1*len(grouped['year'].unique())))
axs2 = [ax.twinx() for ax in axs]
for i, (name, group) in enumerate(grouped):
    # print(group['year'].unique())
    # group.plot(x='date', y=['chinook'], style='-', ax=axs[i])
    axs[i].plot(group['date'], group['chinook'])
    axs[i].set_title(name)
    axs[i].axvline(pd.Timestamp(f"{name}-{fall_start_month}-{fall_start_day}"), color='r', linestyle='--')
    axs[i].axvline(pd.Timestamp(f"{name}-{fall_end_month}-{fall_end_day}"), color='r', linestyle='--')

    axs[i].set_xlim(pd.Timestamp(f"{name}-01-01"), pd.Timestamp(f"{name}-12-31"))

    # remove x-axis labels
    axs[i].set_xticklabels([])


# # add vertical lines for fall
# for i, year in enumerate(grouped['year'].unique()):
#     year = year[0]
#     plt.axvline(pd.Timestamp(f"{year}-{fall_start_month}-{fall_start_day}"), color='r', linestyle='--', ax=axs[i])
#     plt.axvline(pd.Timestamp(f"{year}-{fall_end_month}-{fall_end_day}"), color='r', linestyle='--', ax=axs[i])

# share x-axis amon


grouped = tda_df.groupby('year')

for i, (name, group) in enumerate(grouped):
    # print(group['year'].unique())
    # group.plot(x='date', y=['chinook'], style='-', ax=axs[i])
    axs[i].plot(group['date'], group['chinook'], color='g')
    axs[i].set_title(name)
    axs[i].axvline(pd.Timestamp(f"{name}-{fall_start_month}-{fall_start_day}"), color='r', linestyle='--')
    axs[i].axvline(pd.Timestamp(f"{name}-{fall_end_month}-{fall_end_day}"), color='r', linestyle='--')

    axs[i].set_xlim(pd.Timestamp(f"{name}-01-01"), pd.Timestamp(f"{name}-12-31"))

    # remove x-axis labels
    axs[i].set_xticklabels([])

# grouped = lyl_df.groupby('year')

# for i, (name, group) in enumerate(grouped):
#     # print(group['year'].unique())
#     # group.plot(x='date', y=['chinook'], style='-', ax=axs[i])
#     axs[i].plot(group['date'], group['chinook'], color='g')
#     axs[i].set_title(name)
#     axs[i].axvline(pd.Timestamp(f"{name}-{fall_start_month}-{fall_start_day}"), color='r', linestyle='--')
#     axs[i].axvline(pd.Timestamp(f"{name}-{fall_end_month}-{fall_end_day}"), color='r', linestyle='--')

#     axs[i].set_xlim(pd.Timestamp(f"{name}-01-01"), pd.Timestamp(f"{name}-12-31"))

#     # remove x-axis labels
#     axs[i].set_xticklabels([])


# for i, (name, group) in enumerate(grouped_temp):
#     axs2[i].plot(group['date'], group['AvgTemp'])


fig.tight_layout()


In [13]:
def migration_stats(df, date_col='date', fish_col='chinook', start_month=8, start_day=1, end_month=11, end_day=15):
    df[date_col] = pd.to_datetime(df[date_col])

    df['year'] = df[date_col].dt.year
    df['month'] = df[date_col].dt.month
    df['day'] = df[date_col].dt.day

    df = df.dropna(subset=[fish_col])

    # group bon_df by year and plot
    grouped = df.groupby('year')

    stats_df = pd.DataFrame(columns=['year', 'start_date', 'end_date', 'total_fish', 'daily_mean', 'max_count', '5th_percentile', '5th_date', '25th_percentile', '25th_date', '50th_percentile', '50th_date', '75th_percentile', '75th_date', '95th_percentile', '95th_date'])

    for name, group in grouped:
        stats_dict = {}
        run_start = pd.Timestamp(f"{name}-{start_month}-{start_day}")
        run_end = pd.Timestamp(f"{name}-{end_month}-{end_day}")

        run_df = group[(group[date_col] >= run_start) & (group[date_col] <= run_end)]
        run_df = run_df.sort_values(by=date_col) # sort run_df by date
        
        total_fish = run_df[fish_col].sum()
        # daily mean which is the total fish divided by the number of days from start to end
        daily_mean = total_fish / (run_end - run_start).days
        max_count = run_df[fish_col].max()


        stats_dict['year'] = name
        stats_dict['start_date'] = run_start
        stats_dict['end_date'] = run_end
        stats_dict['total_fish'] = total_fish
        stats_dict['daily_mean'] = daily_mean
        stats_dict['max_count'] = max_count

        # TODO: find a better way to calculate percentiles
        fish_sum = 0
        for i, row in run_df.iterrows():
            fish_sum += row[fish_col]
            
            if fish_sum >= total_fish * 0.05:
                stats_dict['5th_percentile'] = fish_sum
                stats_dict['5th_date'] = row[date_col]
                break

        fish_sum = 0

        for i, row in run_df.iterrows():
            fish_sum += row[fish_col]
            
            if fish_sum >= total_fish * 0.25:
                stats_dict['25th_percentile'] = fish_sum
                stats_dict['25th_date'] = row[date_col]
                break

        fish_sum = 0

        for i, row in run_df.iterrows():
            fish_sum += row[fish_col]
            
            if fish_sum >= total_fish * 0.50:
                stats_dict['50th_percentile'] = fish_sum
                stats_dict['50th_date'] = row[date_col]
                break

        fish_sum = 0

        for i, row in run_df.iterrows():
            fish_sum += row[fish_col]
            
            if fish_sum >= total_fish * 0.75:
                stats_dict['75th_percentile'] = fish_sum
                stats_dict['75th_date'] = row[date_col]
                break

        fish_sum = 0

        for i, row in run_df.iterrows():
            fish_sum += row[fish_col]
            
            if fish_sum >= total_fish * 0.95:
                stats_dict['95th_percentile'] = fish_sum
                stats_dict['95th_date'] = row[date_col]
                break

        stats_df = pd.concat([stats_df, pd.DataFrame(stats_dict, index=[0])])

    return stats_df
                



In [16]:
bon_stats_df = migration_stats(bon_df, start_month=8, start_day=1, end_month=11, end_day=15)
tda_stats_df = migration_stats(tda_df, start_month=8, start_day=4, end_month=10, end_day=31)
lyl_stats_df = migration_stats(lyl_df, start_month=8, start_day=1, end_month=11, end_day=15)

bon_stats_df['duration'] = (bon_stats_df['95th_date'] - bon_stats_df['5th_date']).dt.days
tda_stats_df['duration'] = (tda_stats_df['95th_date'] - tda_stats_df['5th_date']).dt.days
lyl_stats_df['duration'] = (lyl_stats_df['95th_date'] - lyl_stats_df['5th_date']).dt.days

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(15, 15))
axs = axs.flatten()

boxes = []
for i, row in bon_stats_df.iterrows():
    
    boxes.append(
        {
            "label": f"{row['year']}",
            "whislo": (row['5th_date'] -row["start_date"]).days,  # Bottom whisker position
            "q1": (row['25th_date'] -row["start_date"]).days,  # First quartile (25th percentile)
            "med": (row['50th_date'] -row["start_date"]).days,  # Median         (50th percentile)
            "q3": (row['75th_date'] -row["start_date"]).days,  # Third quartile (75th percentile)
            "whishi": (row['95th_date'] -row["start_date"]).days,  # Top whisker position
            "fliers": [],  # Outliers
        }
    )

axs[0].bxp(boxes, showfliers=False, vert=False);
axs[0].set_title("Chinook Fall Migration Timing")

# plot total fish count
axs[1].barh(bon_stats_df['year'], bon_stats_df['total_fish'])
axs[1].set_title("Total Fish Count")
axs[1].set_ylim(1998.5, 2024.5)

# plot max count
axs[2].barh(bon_stats_df['year'], bon_stats_df['max_count'])
axs[2].set_title("Max Fish Count")
axs[2].set_ylim(1998.5, 2024.5)

boxes = []
for i, row in tda_stats_df.iterrows():
    
    boxes.append(
        {
            "label": f"{row['year']}",
            "whislo": (row['5th_date'] -row["start_date"]).days,  # Bottom whisker position
            "q1": (row['25th_date'] -row["start_date"]).days,  # First quartile (25th percentile)
            "med": (row['50th_date'] -row["start_date"]).days,  # Median         (50th percentile)
            "q3": (row['75th_date'] -row["start_date"]).days,  # Third quartile (75th percentile)
            "whishi": (row['95th_date'] -row["start_date"]).days,  # Top whisker position
            "fliers": [],  # Outliers
        }
    )

axs[3].bxp(boxes, showfliers=False, vert=False);
axs[3].set_title("Chinook Fall Migration Timing")

# plot total fish count
axs[4].barh(tda_stats_df['year'], tda_stats_df['total_fish'])
axs[4].set_title("Total Fish Count")
axs[4].set_ylim(1998.5, 2024.5)

# plot max count
axs[5].barh(tda_stats_df['year'], tda_stats_df['max_count'])
axs[5].set_title("Max Fish Count")
axs[5].set_ylim(1998.5, 2024.5)

fig.tight_layout()


In [18]:
bon_tda_temp = pd.read_csv(proj_dir / "Code/notebooks/EDA/bon-tda.csv")
bon_tda_temp['date'] = pd.to_datetime(bon_tda_temp['Date'])
bon_tda_temp = bon_tda_temp[bon_tda_temp['date'].dt.year >= 1999].copy()

bon_tda_temp['year'] = bon_tda_temp['date'].dt.year
bon_tda_temp['month'] = bon_tda_temp['date'].dt.month
bon_tda_temp['day'] = bon_tda_temp['date'].dt.day

fish_dif = bon_stats_df['total_fish'] - tda_stats_df['total_fish']
fish_dif_percent = (bon_stats_df['total_fish'] - tda_stats_df['total_fish']) / bon_stats_df['total_fish'] * 100

lyl_percent = lyl_stats_df['total_fish'] / bon_stats_df['total_fish'] * 100

In [ ]:
# lyl_stats_df.plot(x='year', y='total_fish', style='-')
lyl_stats_df

In [ ]:
# lyl_stats_df.plot(x='year', y='total_fish', style='-')
lyl_stats_df

In [61]:
grouped_temp = bon_tda_temp.groupby('year')

In [ ]:
grouped_temp['year'].unique()

In [ ]:
# for each year, plot the temperature from August 1 to November 15
fig, axs = plt.subplots(len(grouped_temp['year'].unique()), 1, figsize=(10, 1*len(grouped_temp['year'].unique())))

for i, (name, group) in enumerate(grouped_temp):
    axs[i].plot(group['date'], group['AvgTemp'])
    axs[i].set_title(name)
    axs[i].axvline(pd.Timestamp(f"{name}-{fall_start_month}-{fall_start_day}"), color='r', linestyle='--')
    axs[i].axvline(pd.Timestamp(f"{name}-{fall_end_month}-{fall_end_day}"), color='r', linestyle='--')

    axs[i].set_xlim(pd.Timestamp(f"{name}-01-01"), pd.Timestamp(f"{name}-12-31"))

    # remove x-axis labels
    axs[i].set_xticklabels([])

fig.tight_layout()

In [ ]:
# for each year, plot the temperature from August 1 to November 15
fig, ax = plt.subplots(figsize=(10, 5))

years = []
mean_fall_temp = []

for i, (name, group) in enumerate(grouped_temp):

    group['date'] = pd.to_datetime(group['date'])
    group = group[(group['date']>f"{name}-{fall_start_month}-{fall_start_day}") & (group['date']<f"{name}-{fall_end_month}-{fall_end_day}")].copy()
    group['year'] = group['date'].dt.year
    group['month'] = group['date'].dt.month 
    group['day'] = group['date'].dt.day

    group['time_elapsed'] = (group['date'] - pd.Timestamp(f"{name}-{fall_start_month}-{fall_start_day}")).dt.days

    if not group.empty:
        ax.scatter(name, group['AvgTemp'].mean(), label=name)
        years.append(name)
        mean_fall_temp.append(group['AvgTemp'].mean())

        # print(group)

    # # create a new date column that is this_year-month-day
    # group['modified_date'] = pd.to_datetime(f"2020-{group['month']}-{group['day']}")
    # ax.plot(group['modified_date'], group['AvgTemp'], label=name)



    # # ax.axvline(pd.Timestamp(f"{name}-{fall_start_month}-{fall_start_day}"), color='r', linestyle='--')
    # # ax.axvline(pd.Timestamp(f"{name}-{fall_end_month}-{fall_end_day}"), color='r', linestyle='--')

    # # ax.set_xlim(pd.Timestamp(f"{name}-01-01"), pd.Timestamp(f"{name}-12-31"))

    # # remove x-axis labels
    # ax.set_xticklabels([])

fig.tight_layout()

In [ ]:
# plot the fish count difference between BON and TDA
fig, axs = plt.subplots(3, 1, figsize=(10, 7.5), sharex=True)
axs[0].bar(bon_stats_df['year'], fish_dif_percent)
axs[0].set_title("Fish Count Difference (%)")
axs[0].set_ylabel("Percentage difference")

axs[1].bar(bon_stats_df['year'], fish_dif)
axs[1].set_title("Fish Count Difference")
axs[1].set_ylabel(" Count")

axs[2].plot(years, mean_fall_temp)
axs[2].set_title("Mean Fall Temperature")

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()
# plot duration and total fish annotated with the duration
ax.scatter(stats_df['duration'], stats_df['total_fish'])
for i, row in stats_df.iterrows():
    ax.annotate(row['year'], (row['duration'], row['total_fish']))

# ax.scatter(stats_df['duration'], stats_df['max_count'], color='r')
# ax.scatter((stats_df['5th_date'] -stats_df["start_date"]).dt.days, stats_df['total_fish'], color='b')
# ax.set_ylim(0, .8e6)


In [ ]:
fig, ax = plt.subplots()
# plot duration and total fish annotated with the duration
# ax.scatter(stats_df['duration'], stats_df['total_fish'])
# for i, row in stats_df.iterrows():
#     ax.annotate(row['duration'], (row['duration'], row['total_fish']))

# ax.scatter(stats_df['duration'], stats_df['max_count'], color='r')
m = ax.scatter((stats_df['95th_date'] -stats_df["start_date"]).dt.days,(stats_df['5th_date'] -stats_df["start_date"]).dt.days, c=stats_df['total_fish'])
# add colorbar
cbar = fig.colorbar(m)
